# Request TemperatureHeure

In [1]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///db.sqlite')
connection = engine.connect()

In [2]:
request = """ 
    SELECT TemperatureHeure.temperature, TemperatureHeure.departement_code, TemperatureHeure.date
    FROM TemperatureHeure 
    WHERE TemperatureHeure.departement_code=59
"""

temperature = pd.read_sql(request, connection,index_col="date")
temperature["temperature"]=temperature["temperature"].astype(float)
temperature.index = pd.to_datetime(temperature.index)

In [8]:
temperature.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 63302 entries, 2016-12-31 21:00:00+00:00 to 2021-12-10 06:00:00+00:00
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   temperature       63302 non-null  float64
 1   departement_code  63302 non-null  object 
dtypes: float64(1), object(1)
memory usage: 1.4+ MB


In [17]:
temp_group_by = temperature.groupby('departement_code').resample('D').agg("mean")
temp_group_by

/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_54077/1187376375.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  temp_group_by = temperature.groupby('departement_code').resample('D').agg("mean")


temperature
departement_code date                                  
59               2016-12-10 00:00:00+00:00     8.366667
                 2016-12-11 00:00:00+00:00     7.825000
                 2016-12-12 00:00:00+00:00     4.837500
                 2016-12-13 00:00:00+00:00     6.200000
                 2016-12-14 00:00:00+00:00     5.650000
...                                                 ...
                 2021-12-27 00:00:00+00:00     8.337500
                 2021-12-28 00:00:00+00:00    10.287500
                 2021-12-29 00:00:00+00:00    10.875000
                 2021-12-30 00:00:00+00:00    14.050000
                 2021-12-31 00:00:00+00:00    12.525000

[1848 rows x 1 columns]

In [18]:
temp_group_by = temp_group_by.reset_index()


In [15]:
temp_group_by.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1848 entries, 0 to 1847
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   departement_code  1848 non-null   object 
 1   date              1848 non-null   object 
 2   temperature       131 non-null    float64
dtypes: float64(1), object(2)
memory usage: 43.4+ KB


# Request Lost_Items

In [19]:
request = """ 
    SELECT LostItem.code_uic_gare_origine , COUNT(*) 
    FROM LostItem 
    GROUP BY LostItem.code_uic_gare_origine
"""

item_gb = pd.read_sql(request, connection)

In [22]:
item_gb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   code_uic_gare_origine  178 non-null    object
 1   COUNT(*)               179 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.9+ KB


In [26]:
request = """ 
    SELECT * 
    FROM Gare 
"""

gare_gb = pd.read_sql(request, connection)

In [30]:
gare_gb["code_uic"].value_counts().sort_values()

0087726216    1
0087775122    1
0087753541    1
0087775247    1
0087784843    1
             ..
0087584458    1
0087249789    1
0087734087    2
0087271007    2
0087645119    2
Name: code_uic, Length: 3210, dtype: int64

In [28]:
gare_gb.sort_values(by="code_uic")

,id,code_uic,nom_gare,code_postal,code_commune,code_departement,departement,longitude,latitude
1256,5e6c947d7b4656435bd51d98b812591382632924,0087000028,Sevenans,90400,094,90,Territoire de Belfort,NaN,NaN
2588,61fd0a21f177438f872859fb149fa8cc6e7c2863,0087000032,Le Mans hôpital-université,72000,None,None,None,NaN,NaN
1145,f90a4a67faa88f3f424e6967e09cc0653f642e04,0087001479,Aéroport Charles de Gaulle 2 TGV,95700,527,95,Val-d'Oise,2.570892,49.003652
1896,a3a687d0751bbec6a7ba16870933438ae073b622,0087009696,La Douzillère,37300,122,37,Indre-et-Loire,0.652824,47.338817
1895,11e9af039e38eeb54620ae596abf947a26bf423c,0087109306,Lille CHR,59000,350,59,Nord,3.034320,50.614320
...,...,...,...,...,...,...,...,...,...
1281,c1eb0555ec810840ed1866aaf02bfb376a8029c1,0087784892,Bena Fanes,66760,066,66,Pyrénées-Orientales,1.916726,42.458078
664,ad72975073065b6bbd0b4a5e7f3299972f5ab6b3,0087785006,Cerbère,66290,048,66,Pyrénées-Orientales,3.163403,42.441773
3013,5b54fe3746858c7d41facbfe1e90115d73fe8a41,0087981902,Avignon TGV,84000,007,84,Vaucluse,4.786079,43.921586
663,c916d3f5dbd75d76305e7aa537e67daeb8f239e9,0087988709,Remise à Jorelle,93140,010,93,Seine-Saint-Denis,2.487751,48.893170


In [24]:
le_merge = item_gb.merge(gare_gb, how='left',left_on="code_uic_gare_origine", right_on="code_uic" )

In [31]:
le_merge["code_departement"].unique()

array([nan, '75', '10', '54', '88', '51', '68', '90', '67', '57', '59',
       '95', '60', '62', '80', '84', '13', '27', '76', '14', '61', '35',
       '29', '56', '53', '44', '49', '17', '85', '45', '37', '41', '86',
       '18', '33', '16', '24', '47', '87', '36', '31', '11', '65', '64',
       '34', '58', '03', '21', '25', '69', '71', '42', '63', '73', '01',
       '74', '38', '83', '06', '26', '05', '30', '66'], dtype=object)